# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd
data=pd.read_csv('/datasets/data.csv')
#Вызываем общие данные таблицы методом info()
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.1 Заполнение пропусков

In [2]:
display(data.head(30))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
#При просмотре были обнаружены пропущенные данные типа NaN.
#Вычисляем долю пропущенных значений в колонках days_employed и total_income, 
#количество строк берем из общих данных таблицы
print('Доля  пропущенные значений days_employed {:.0%}'.format(data['days_employed'].isna().sum()/19351))
print('Доля  пропущенные значений total_income {:.0%}'.format(data['total_income'].isna().sum()/19351))
print()

#Вычисляем медианну для колонки days_employed 
median_days_employed=data['days_employed'].median()
print('Медианное значение days_employed', median_days_employed)
print()
#Заменяем пропущенные значения в колонке days_employed методом fillna()
data['days_employed']=data['days_employed'].fillna(median_days_employed)
print()
#Посмотрим какие есть уникальные зачения и их количество в столбике income_type
print(data['income_type'].value_counts()) 
print()
#Найдем медианные уровни дохода для каждого типа занятости
medians_total_income=data.groupby('income_type')['total_income'].median()
print(medians_total_income)
print()
#Заменим пропущенные значения ежемесячного дохода на медианы по типам занятости.
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'сотрудник'), 'total_income'] = medians_total_income[6]
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'компаньон'), 'total_income'] = medians_total_income[3]
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'пенсионер'), 'total_income'] = medians_total_income[4]
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'госслужащий'), 'total_income'] = medians_total_income[2]
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'предприниматель'), 'total_income'] = medians_total_income[5]
#В типах занятости студент,  декрет и безработные нет пропущенных значений, мы их не трогаем  
    
#Проверяем наличие пропущеных значений в таблице
print(data.isna().sum())




Доля  пропущенные значений days_employed 11%
Доля  пропущенные значений total_income 11%

Медианное значение days_employed -1203.369528770489


сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


Появление этих ошибок связано как с людьми, не правильно были записаны данные, так и не правильно при копировании были сохранены данные.
Получение медианного значения лучше, потому что дает более объективную информацию, чем среднее значение, (в колонке присутствуют отрицательные числа, и при подсчете медианы не имеет значение какого знака были данные).

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [4]:
#Находим аномалии в колонке children
print(data['children'].value_counts())
print()
#С помощью функции replace() заменяем данные 
data['children']=data['children'].replace(-1, 1)
data['children']=data['children'].replace(20, 2)
#Проверяем изменения
print(data['children'].value_counts())
print()

#Находим аномалии в колонке days_employed
print(data['days_employed'].value_counts())
#Заменяем отрицательные значения на положительные методом abs(), т.к. трудовой стаж не может иметь отрицательное значение
data['days_employed']=data['days_employed'].abs()
print()

#Проверяем наши изменения
print(data['days_employed'].value_counts())


#Проверка  в остальных колонках показала очень не значительные ошибки, ими можно пренебречь.







 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

-1203.369529      2175
-327.685916          1
-4399.664923         1
-4122.460569         1
-2828.237691         1
                  ... 
-2146.884040         1
-881.454684          1
-794.666350          1
 372337.985304       1
-3382.113891         1
Name: days_employed, Length: 19351, dtype: int64

1203.369529      2175
142.276217          1
329951.594147       1
886.253127          1
2539.534295         1
                 ... 
1394.302246         1
2325.720832         1
4086.407828         1
1259.497032         1
1636.419775         1
Name: days_employed, Length: 19351, dtype: int64


### Шаг 2.3. Изменение типов данных.

In [5]:
#Заменяем вещественный тип даных на целочисленный в колонке total_income 

data['total_income']=data['total_income'].astype('int')
print(data['total_income'].head(10))

0    253875
1    112080
2    145885
3    267628
4    158616
5    255763
6    240525
7    135823
8     95856
9    144425
Name: total_income, dtype: int64


### Шаг 2.4. Удаление дубликатов.

In [6]:
#Проверяем наличие дубликатов в таблице
print(data.duplicated().sum())






54


In [7]:
#Проверяем наши  изменения
data=data.drop_duplicates().reset_index(drop=True)
print(data.duplicated().sum())


0


Для поиска дубликатов использовали метод duplicated(), удаляли дубликаты 
при помощи метода drop_duplicates()

In [8]:
#Для изменения регистра используем метод str.lower()
data['education']=data['education'].str.lower()
print(data['education'].head(30))

0                  высшее
1                 среднее
2                 среднее
3                 среднее
4                 среднее
5                  высшее
6                  высшее
7                 среднее
8                  высшее
9                 среднее
10                 высшее
11                среднее
12                среднее
13    неоконченное высшее
14                 высшее
15                среднее
16                среднее
17                 высшее
18                среднее
19                среднее
20                среднее
21                среднее
22                среднее
23                 высшее
24                среднее
25                среднее
26                среднее
27                 высшее
28                 высшее
29                среднее
Name: education, dtype: object


### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [9]:
#Создаем новые талицы
data_education=data[['education_id','education']]
data_family_status=data[['family_status_id', 'family_status']]
#Удаляем колонки education и family_status методом рор()
data.pop('education')
data.pop('family_status')







0         женат / замужем
1         женат / замужем
2         женат / замужем
3         женат / замужем
4        гражданский брак
               ...       
21466    гражданский брак
21467     женат / замужем
21468    гражданский брак
21469     женат / замужем
21470     женат / замужем
Name: family_status, Length: 21471, dtype: object

### Шаг 2.6. Категоризация дохода.

Создаем колонку total_income_category, применяя метод apply()

In [10]:
def total_income_category (total_income):
    if total_income<=30000:
        return 'E'
    if 30001<=total_income<=50000:
        return 'D'
    if 50001<=total_income<=200000:
        return 'C'
    if 200001<=total_income<=1000000:
        return 'B'
    return 'A'

data['total_income_category']=data['total_income'].apply(total_income_category)
print(data['total_income_category'].value_counts())

C    16032
B     5042
D      350
A       25
E       22
Name: total_income_category, dtype: int64


### Шаг 2.7. Категоризация целей кредита.

Создаем функцию total_income_category (), используя метод apply()

In [11]:
def purpose_category (purpose):
    if 'автомобил' in purpose:                                                                  
        return 'операции с автомобилем'
    if 'образовани' in purpose:
        return 'получение образования'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    return 'операции с недвижимостью'

data['purpose_category']=data['purpose'].apply(purpose_category)
#Проверяем результаты нашей деятельности, вызываем дисплей, потому что он более удобно показывает информацию:
display(data.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


### Ответы на вопросы.

##### Вопрос 1:Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [12]:
#Создаем сводную таблицу по данным колонок purpose_category и debt 
credit_children=data.pivot_table(index='children', columns='debt', values='dob_years',  aggfunc='count')
#Добавляем новую колонку в сводной таблице, где посчитываем процент невозврата в срок
credit_children['ratio']=credit_children[1]/(credit_children[0]+credit_children[1])*100
display(credit_children.sort_values(by='ratio', ascending=False))

debt,0,1,ratio
children,,,
4,37.0,4.0,9.756098
2,1926.0,202.0,9.492481
1,4411.0,445.0,9.163921
3,303.0,27.0,8.181818
0,13044.0,1063.0,7.535266
5,9.0,NaN,NaN


In [13]:
# КОД РЕВЬЮЕРА

data.groupby('children')['debt'].agg(['count','sum','mean'])

,count,sum,mean
children,,,
0,14107,1063,0.075353
1,4856,445,0.091639
2,2128,202,0.094925
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


In [14]:
# КОД РЕВЬЮЕРА

def my_mean(x): return x.mean()*100

# В кач-ве агрегирования можно любую функцию брать.
data.groupby('children')['debt'].agg(['count', 'sum', my_mean])

,count,sum,my_mean
children,,,
0,14107,1063,7.535266
1,4856,445,9.163921
2,2128,202,9.492481
3,330,27,8.181818
4,41,4,9.756098
5,9,0,0.000000


In [15]:
# КОД РЕВЬЮЕРА

def my_mean(x): return '{:.2%} '.format(x.mean())

data.groupby('children')['debt'].agg(['count', 'sum', my_mean])

,count,sum,my_mean
children,,,
0,14107,1063,7.54%
1,4856,445,9.16%
2,2128,202,9.49%
3,330,27,8.18%
4,41,4,9.76%
5,9,0,0.00%


##### Вопрос 2:Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [16]:
#Создаем сводную таблицу по данным колонок family_status_id и debt 
credit_family_status_id=data.pivot_table(index='family_status_id', columns='debt', values='dob_years',  aggfunc='count')
#Добавляем новую колонку в сводной таблице, где подсчитываем процент невозврата в срок
credit_family_status_id['ratio']=credit_family_status_id[1]/(credit_family_status_id[0]+credit_family_status_id[1])*100
display(credit_family_status_id.sort_values(by='ratio', ascending=False))




debt,0,1,ratio
family_status_id,,,
4,2536,274,9.750890
1,3775,388,9.320202
0,11413,931,7.542126
3,1110,85,7.112971
2,896,63,6.569343


##### Вопрос 3:Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [17]:
#Создаем сводную таблицу по данным колонок total_income_category и debt 
credit_total_income_category=data.pivot_table(index='total_income_category', columns='debt', values='dob_years',  aggfunc='count')
#Добавляем новую колонку в сводной таблице, где подсчитываем процент невозврата в срок
credit_total_income_category['ratio']=credit_total_income_category[1]/(credit_total_income_category[0]+credit_total_income_category[1])*100
display(credit_total_income_category.sort_values(by='ratio', ascending=False))




debt,0,1,ratio
total_income_category,,,
E,20,2,9.090909
C,14672,1360,8.483034
A,23,2,8.000000
B,4686,356,7.060690
D,329,21,6.000000


##### Вопрос 4:Как разные цели кредита влияют на его возврат в срок?

In [18]:
#Создаем сводную таблицу по данным колонок purpose_category и debt 
credit_purpose_category=data.pivot_table(index='purpose_category', columns='debt', values='dob_years',  aggfunc='count')
#Добавляем новую колонку в сводной таблице, где посчитываем процент невозврата в срок
credit_purpose_category['ratio']=credit_purpose_category[1]/(credit_purpose_category[0]+credit_purpose_category[1])*100
display(credit_purpose_category.sort_values(by='ratio', ascending=False))

debt,0,1,ratio
purpose_category,,,
операции с автомобилем,3905,403,9.354689
получение образования,3644,370,9.217738
проведение свадьбы,2149,186,7.965739
операции с недвижимостью,10032,782,7.231367


##### Вывод 1:

По сводной та,лице видно, что вероятность не возврата кредита больше у клиентов, у которых есть дети, а меньше у тех, кто не имеет детей.

##### Вывод 2:

По сводной та,лице видно, что вероятность не возврата кредита больше у клиентов, которые женаты или замужем, чем у клиентов не имеющих семейного положения.

##### Вывод 3:

По сводной та,лице видно, что вероятность не возврата кредита больше у клиентов с наименьшим доходом.

##### Вывод 4:

По сводной та,лице видно, что самыми отвественными клиентами являются люди, которые берут кредит на недвижмость, а безотвественными те, которые берут кредит на машину.

## Общий вывод:

При работе с данными были заполнены пропущенные значения, удалены дубликаты и выделены категории для целей кредита, группы по возрасту и образованию. Применив сводные таблицы, я оценил, какие критерии влияют на возврат кредита. Дополнительный анализ может помочь более точно предсказать вероятность вылпаты кредита в срок.